# Rafael Andreolli Prochnow RA176059

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import distance
import cmath
from IPython.display import clear_output
import time

# Parte 1

In [2]:
# Tabela apresentada 
d = {'Alternativas': ['A1', 'A2', 'A3', 'A4'],
     'Preço': [18000, 20500, 24700, 22800],
     'Conforto': [3, 5, 5, 10], 
     'Potência': [8, 5, 4, 7]
    }
# Critério
crit_max = [False, True, True]
frame = pd.DataFrame(data=d)
# Pesos: w
w = [0.5, 0.2, 0.3]

In [3]:
frame

,Alternativas,Preço,Conforto,Potência
0,A1,18000,3,8
1,A2,20500,5,5
2,A3,24700,5,4
3,A4,22800,10,7


In [4]:
def usual(frame, coluna, i, j, matriz, crit_max):
    q1 = frame.loc[j, frame.columns[coluna]]
    q2 = frame.loc[i, frame.columns[coluna]]
    if crit_max[coluna-1]:
        if q1-q2 == 0:
            matriz[i, j] = 0
        elif q1-q2 > 0:
            matriz[i, j] = 0
        elif q1-q2 < 0:
            matriz[i, j] = 1
    else:
        if q1-q2 == 0:
            matriz[i, j] = 0
        elif q1-q2 > 0:
            matriz[i, j] = 1
        elif q1-q2 < 0:
            matriz[i, j] = 0
    return matriz

In [5]:
def metodo_promethee(frame, peso, preferencia, crit_max, promethee):
    for i in frame.columns:
        if i == 'Ordenamento_saw':
            frame.drop(columns=['Ordenamento_saw'], inplace=True)
        if i == 'Ordenamento_topsis':
            frame.drop(columns=['Ordenamento_topsis'], inplace=True)
        if i == 'Promethee_II':
            frame.drop(columns=['Promethee_II'], inplace=True)
        else:
            pass
    
    # juntar todas as respostas
    resultado = []
    for k in range(1, len(frame.columns)):
        # Tamanho da Matriz de decisão
        tamanho = len(frame)

        # Matriz da comparação par a par dos critérios
        matriz = np.zeros((tamanho, tamanho), dtype=float)
        # Resultado da sua matriz 
        sua_matriz = []
        if preferencia == 'usual':
            for i in range(tamanho):
                for j in range(tamanho):
                    # Escolher qual a Matriz de Preferencia
                    matriz = usual(frame, k, i, j, matriz, crit_max)
            resultado.append(matriz)

    # Matriz M 
    # número de vitórias
    matriz = np.sum([kk*k for kk, k in zip(resultado,peso)], axis=0)
    
    # Matriz do Fluxo Positivo e Negativo
    matriz_fluxo = [[np.sum(_)/(len(_)-1) for _ in matriz], [np.sum(_)/(len(_)-1) for _ in matriz.T]]

    if promethee == 2:
        resultado = list(matriz_fluxo[0][_] - matriz_fluxo[1][_] for _ in range(len(matriz_fluxo[0])))
        
        # Passo 6 Odenar as alternativas de acordo com os índeces da ordenação crescente do valor r
        frame['Promethee_II'] = resultado
        final = list(frame.sort_values(by='Promethee_II', ascending=False)[frame.columns[0]])
        ordem = 1
        for x in final:
            print(f'{ordem}° Lugar -->', x)
            ordem += 1

    return frame

In [6]:
resultado = metodo_promethee(frame=frame, peso=w, preferencia='usual', crit_max=crit_max, promethee=2)
resultado

1° Lugar --> A1
2° Lugar --> A4
3° Lugar --> A2
4° Lugar --> A3


,Alternativas,Preço,Conforto,Potência,Promethee_II
0,A1,18000,3,8,0.600000
1,A2,20500,5,5,0.066667
2,A3,24700,5,4,-0.800000
3,A4,22800,10,7,0.133333


In [7]:
# Tabela apresentada 
d = {'Alternativas': ['Bira', 'Vanessa', 'José Gomes', 'Flavia', 'Gouglas', 'Cleide', 'Dicia', 'Ranualdo', 'Mauro', 'Peru'],
     'Ticket Médio': [25, 25, 29, 24, 25, 24, 28, 24, 26, 41],
     'Qualidade do Atendimento': [6.7, 7.0, 5.7, 6.3, 7.5, 8.7, 8.3, 7.3, 7.2, 8.5], 
     'Detalhes Pessoais': [8.0, 7.0, 6.0, 6.3, 7.3, 8.2, 8.0, 7.7, 7.3, 7.7], 
     'Relacionamento interpessoal': [8.3, 7.3, 7.0, 7.0, 8.3, 9.0, 8.7, 7.3, 8.2, 8.3] 
    }
# Critério MAX e MIN
crit_max = [True, True, True, True]
# Pesos: w
w = [0.25, 0.25, 0.25, 0.25]
df = pd.DataFrame(data=d)
df

,Alternativas,Ticket Médio,Qualidade do Atendimento,Detalhes Pessoais,Relacionamento interpessoal
0,Bira,25,6.7,8.0,8.3
1,Vanessa,25,7.0,7.0,7.3
2,José Gomes,29,5.7,6.0,7.0
3,Flavia,24,6.3,6.3,7.0
4,Gouglas,25,7.5,7.3,8.3
5,Cleide,24,8.7,8.2,9.0
6,Dicia,28,8.3,8.0,8.7
7,Ranualdo,24,7.3,7.7,7.3
8,Mauro,26,7.2,7.3,8.2
9,Peru,41,8.5,7.7,8.3


In [8]:
resultado = metodo_promethee(frame=df, peso=w, preferencia='usual', crit_max=crit_max, promethee=2)
resultado

1° Lugar --> Dicia
2° Lugar --> Peru
3° Lugar --> Cleide
4° Lugar --> Bira
5° Lugar --> Gouglas
6° Lugar --> Mauro
7° Lugar --> Ranualdo
8° Lugar --> Vanessa
9° Lugar --> José Gomes
10° Lugar --> Flavia


,Alternativas,Ticket Médio,Qualidade do Atendimento,Detalhes Pessoais,Relacionamento interpessoal,Promethee_II
0,Bira,25,6.7,8.0,8.3,0.083333
1,Vanessa,25,7.0,7.0,7.3,-0.361111
2,José Gomes,29,5.7,6.0,7.0,-0.527778
3,Flavia,24,6.3,6.3,7.0,-0.805556
4,Gouglas,25,7.5,7.3,8.3,0.083333
5,Cleide,24,8.7,8.2,9.0,0.555556
6,Dicia,28,8.3,8.0,8.7,0.638889
7,Ranualdo,24,7.3,7.7,7.3,-0.222222
8,Mauro,26,7.2,7.3,8.2,-0.027778
9,Peru,41,8.5,7.7,8.3,0.583333


In [9]:
aaaaaaaaa

NameError: name 'aaaaaaaaa' is not defined

In [ ]:
def estrutura_p(preferencia_p, matriz_fluxo, i, j):
    if (matriz_fluxo[0][i] > matriz_fluxo[0][j]) & (matriz_fluxo[1][i] < matriz_fluxo[1][j]):
        preferencia_p[i, j] = 1
    elif (matriz_fluxo[0][i] == matriz_fluxo[0][j]) & (matriz_fluxo[1][i] < matriz_fluxo[1][j]):
        preferencia_p[i, j] = 1
    elif (matriz_fluxo[0][i] > matriz_fluxo[0][j]) & (matriz_fluxo[1][i] == matriz_fluxo[1][j]):
        preferencia_p[i, j] = 1
    else:
        preferencia_p[i, j] = 0
    return preferencia_p

In [ ]:
def estrutura_i(preferencia_i, matriz_fluxo, i, j):
    if (matriz_fluxo[0][i] == matriz_fluxo[0][j]) & (matriz_fluxo[1][i] == matriz_fluxo[1][j]):
        preferencia_i[i, j] = 1
    else:
        preferencia_i[i, j] = 0
    return preferencia_i

In [ ]:
def estrutura_j(preferencia_j, matriz_fluxo, i, j):
    if (matriz_fluxo[0][i] > matriz_fluxo[0][j]) & (matriz_fluxo[1][i] > matriz_fluxo[1][j]):
        preferencia_j[i, j] = 1
    elif (matriz_fluxo[0][i] < matriz_fluxo[0][j]) & (matriz_fluxo[1][i] < matriz_fluxo[1][j]):
        preferencia_j[i, j] = 1
    else:
        preferencia_j[i, j] = 0
    return preferencia_j

In [ ]:
# criar as tabelas como apresentado na aula e depois fazer algo e depois o ordenamento e sei lá mais o que 
preferencia_p = np.zeros((len(matriz_fluxo[0]), len(matriz_fluxo[0])), dtype=float)
preferencia_i = np.zeros((len(matriz_fluxo[0]), len(matriz_fluxo[0])), dtype=float)
preferencia_j = np.zeros((len(matriz_fluxo[0]), len(matriz_fluxo[0])), dtype=float)
for i in range(len(matriz_fluxo[0])):
    for j in range(len(matriz_fluxo[0])):
        preferencia_p = estrutura_p(preferencia_p, matriz_fluxo, i, j)
        preferencia_i = estrutura_i(preferencia_i, matriz_fluxo, i, j)
        preferencia_j = estrutura_j(preferencia_j, matriz_fluxo, i, j)

In [ ]:
preferencia_p

In [ ]:
preferencia_i

In [ ]:
preferencia_j

# Parte 2